<a href="https://colab.research.google.com/github/mayuneko-re/notebook/blob/master/colab/PT_Flash_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PTフラッシュ計算

PTフラッシュ計算は，あるモル成分Ziの混合物をある温度・圧力条件下で気相と液相に分ける．

**PTフラッシュ計算の流れ**

1.   フィード流体を設定する
2.   最初のK値を推定（Wilsonの式）
3.   K値から、各相の成分を計算（Ratchford-Riceの式, WhitsonのNegative Flash）,
4.   液相・気相それぞれについて、多成分系のEOSを解き（カルダノの方法）、Partial fugacity coefficientを計算
5.   両相のPartial fugacity coefficientからK値を更新
6.   3-5を繰り返し計算。K値の変化が十分小さくなった時点で計算終了。


## Equation of state (EOS)

流体モデルの相挙動は，EOSは業界でよくみられる **Peng-Robinson (PR) EOS** を使用してやってみる．**PR EOS** は以下のように表現される．

$$
p = \frac{RT}{v-b} - \frac{a}{v(v+b)+b(v-b)}
$$

単成分に対するEOSの定数は以下のとおり．

$$
a = 0.45724 \frac{R^2T_c^2}{p_c}\alpha, \ b=0.07780\frac{RT_c}{p_c}, \\
\alpha=\left[1+m \left(1-\sqrt{T/T_c} \right) \right]^2, \ m=0.37464+1.54226\omega-0.26992\omega^2
$$

$\omega$は，acentric factorと呼ばれるパラメータである．

zファクターに関して表式すると以下のようになる．

$$
z^3-(1-B)z^2+(A-3B^2-2B)z-(AB-B^2-B^3)=0
$$

$$
A = a \frac{p}{(RT)^2}, \ B = b \frac{p}{RT}
$$

**Mixtureに関するEOS**

相*j*の混合物に対するPR EOSのパラメータは，**Mixing rule** によって以下のようになる．

$$
A_{mj} = \sum_{i=1}^N \sum_{k=1}^N x_i x_k A_{ik}, \ 
B_{mj} = \sum_{i=1}^N x_i B_i, \
A_{ik} = (1-k_{ik})\sqrt{A_i A_k} 
$$

相*j*中の成分*i*の **Partial fugacity coefficient**, $\phi_ij$ は以下のようになる．

$$
\ln \phi_{ij} = \frac{B_i}{B_{mj}}(Z_{mj}-1)-\ln (Z_{mj}-B_{mj}) 
  + \frac{A_{mj}}{2\sqrt{2}B_{mj}}\left( \frac{B_{i}}{B_{mj}} - \frac{2 \sum_{k=1}^N x_k A_{ik}}{A_{mj}}  \right)
  \ln \left[ \frac{Z_{mj}+(1+\sqrt{2})B_{mj}}{Z_{mj}+(1-\sqrt{2})B_{mj}} \right]
$$

成分iの平衡定数 $K_i$は，気相V・液相LそれぞれのPartial fugacity coefficientを用いて以下のように表される．

$$
K_i = \frac{\phi_{iL}}{\phi_{iV}}
$$


**ライブラリのインポート**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

## クラスの作成

**Componentsクラス**

各成分のPR EOSパラメータを計算し保持するクラス．

In [0]:
class Components():

    R = 8.3142 # Gas constant, kPa-m3/(Kg-mol-K)

    def __init__(self, name, Pc, Tc, omega):
        self.name = name # name of component
        self.Pc = Pc # Critical pressure, BarA
        self.Tc = Tc # Critical temperature degK
        self.omega = omega # Acentric factor

    def PREOS(self, P, T):
        """Calculate Peng-Robinson EOS parameters
        """
        self.P = P # Pressure condition, BarA
        self.T = T # Temperature condition, degK

        # Peng-Robinson EoS parameters
        m = 0.37464 + 1.54226*self.omega -0.26992*self.omega**2
        self.alpha = (1 + m*(1-np.sqrt(T/self.Tc)))**2
        self.a = 0.45724*(self.R*self.Tc)**2*self.alpha/(self.Pc*1e5)
        self.b = 0.07780*self.R*self.Tc/(self.Pc*1e5)
        self.A = self.a * (self.P*1e5) / (self.R*self.T)**2
        self.B = self.b * (self.P*1e5) / (self.R*self.T)
        
        return True

**Mixtureクラス**

多成分系のPR EOSパラメータを計算し保持、PTフラッシュ計算を実施するクラス．

In [0]:
class Mixture():

    R = 8.3142 # Gas constant, kPa-m3/(Kg-mol-K)

    def __init__(self, feed, Zi):
        self.feed = np.array(feed) # Feed composition instance list
        self.Zi = Zi / np.sum(Zi) # Feed molar fraction
        self.set_BIPs() # Set zero BIPs

    def set_BIPs(self, kik=None):
        """Set Binary Interaction Parameters for PR EOS, which is available from literature.
        """
        if kik is None:
            self.kik = np.zeros((self.feed.size, self.feed.size)) # No information
        else:
            self.kik = kik
        

    def PT_flash(self, P, T, verbose=False):
        """PT Flash
        """
        self.P = P # Pressure condition for PT flash, BarA
        self.T = T # Temperature condition for PT flash, degK

        # Initialize PR EOS parameters for each components at P and T condition
        for c in self.feed:
            c.PREOS(self.P, self.T)

        # Initial guess of Ki is made by Wilson equation.
        self.Ki = np.array([(c.Pc/self.P)*np.exp(5.37*(1+c.omega)*(1-c.Tc/self.T)) 
                            for c in self.feed])

        # Mixture parameters are calculated by mixing rules.
        self.Ai = np.array([c.A for c in self.feed])
        self.Bi = np.array([c.B for c in self.feed])
        self.Aik = np.outer(self.Ai,self.Ai)**0.5 * (1-self.kik)
        
        def f(V, Z, Ki):
            f = np.sum( (Z*(Ki-1)) / (1+V*(Ki-1)) )
            return f

        # New values of Ki calculated are again used to estimate V and 
        # thereafter Xi & Yi. Iteration is repeated till there is no further 
        # change in Ki values.
        deltaKi = 10
        tol = 1e-6

        while deltaKi>tol:

            # Relative molar volume in vapor phase
            min = 1/(1-np.max(self.Ki))
            max = 1/(1-np.min(self.Ki))
            min = min + np.abs(min)*1e-6
            max = max - np.abs(max)*1e-6
            self.V = optimize.bisect(f, min, max, args=(self.Zi, self.Ki))
            self.L = 1 - self.V

            # Compositions in liquid phase and vapor phase
            self.Xi = self.Zi / (1+self.V*(self.Ki-1))
            self.Yi = self.Xi * self.Ki

            # Partial fugacity coefficient calculation for Liquid phase and Vapor phase
            PhiL = self.calc_fugacity(self.Xi, P)
            PhiV = self.calc_fugacity(self.Yi, P)

            # Update K
            Kinew = PhiL / PhiV
            deltaKi = np.sum(np.abs(Kinew/self.Ki-1))
            self.Ki = Kinew

        loc = np.where(self.Zi == 0)[0]
        self.Xi = np.insert(self.Xi, loc, 0)
        self.Yi = np.insert(self.Yi, loc, 0)
        self.Ki = np.insert(self.Ki, loc, np.nan)

        if self.V >=1:
            self.phase = 'vapor'
            self.Vact, self.Lact = 1, 0
            self.Xiact, self.Yiact = np.zeros_like(self.Zi), self.Zi
        elif self.V<=0:
            self.phase = 'liquid'
            self.Vact, self.Lact = 0, 1
            self.Xiact, self.Yiact = self.Zi, np.zeros_like(self.Zi)
        else:
            self.phase = 'two-phase'
            self.Vact, self.Lact = self.V, self.L
            self.Xiact, self.Yiact = self.Xi, self.Yi

        if verbose:
            self.print_result()
            

    def calc_fugacity(self, xi, P):

        # Mixture parameters are calculated by mixing rules.
        A = np.sum(self.Aik * xi * xi.reshape(-1, 1))
        B = np.sum(self.Bi * xi)

        Zj = CardanoEOS(A,B) # Solving z factor

        lnPhi = self.Bi/B*(Zj-1) - np.log(Zj-B) \
            - A/(2*np.sqrt(2)*B)*(2*np.dot(self.Aik, xi)/A-self.Bi/B) \
            * np.log( (Zj+(1+np.sqrt(2))*B)/(Zj+(1-np.sqrt(2))*B) )
        Phi = np.exp(lnPhi)

        return Phi


    def print_result(self):
        print('PT Flash calculation converged.')
        print()
        print('                       Feed components : {}'.format([c.name for c in self.feed]))
        print('                    Feed mole fraction : {}'.format(self.Zi))
        print()
        print('PT Flash at {0:.1f} degK and {1:.1f} BarA.'.format(self.T, self.P))
        print()
        print('                                 Phase : {}'.format(self.phase))
        print('Relative mole fraction of liquid phase : {:.4f}'.format(self.Lact))
        print(' Relative mole fraction of vapor phase : {:.4f}'.format(self.Vact))
        print('         Mole fraction in liquid phase : {}'.format(self.Xiact))
        print('          Mole fraction in vapor phase : {}'.format(self.Yiact))
        print('                              K values : {}'.format(self.Ki))


def CardanoEOS(A,B):

    C2 = B-1
    C1 = A - 3*B**2 - 2*B
    C0 = B**3 + B**2 - A*B
    q = C0 - 1/3*C1*C2 + 2/27*C2**3
    p = C1 - C2**2/3
    D = (q/2)**2 + (p/3)**3

    return np.cbrt(-q/2+np.sqrt(D)) + np.cbrt(-q/2-np.sqrt(D)) - C2/3

## 成分のセット

ここでは、**メタン（C1）、ブタン（C4）、デカン（C10）、CO2**の４種を設定した．

In [0]:
# Set components which you need.
methane = Components(name='methane', Pc=46.00155, Tc=190.6, omega=0.008)
butane = Components(name='butane', Pc=37.996875, Tc=425.2, omega=0.193)
decane = Components(name='decane', Pc=21.0756, Tc=617.6, omega=0.49)
co2 = Components(name='co2', Pc=73.7646, Tc=304.2, omega=0.225)

## Mixtureの作成とPHフラッシュ計算



### ケース１：C1-C4-C10系

モル分率を下記のとおりとする．

- C1: 0.6
- C4: 0.1
- C10: 0.3

In [5]:
feed = [methane, butane, decane]
Zi = np.array([0.6, 0.1, 0.3])
kik = np.array([[0     , 0.0133, 0.0422],
                [0.0133, 0     , 0     ],
                [0.0422, 0     , 0     ]
                ])

# Pressure
P = 2000/14.5038 # 2000psia -> BarA

# Temperature
T = (180-32)*5/9+273.15 # 180degF -> degK

m = Mixture(feed, Zi)
m.set_BIPs(kik)
m.PT_flash(P,T,verbose=True)

PT Flash calculation converged.

                       Feed components : ['methane', 'butane', 'decane']
                    Feed mole fraction : [0.6 0.1 0.3]

PT Flash at 355.4 degK and 137.9 BarA.

                                 Phase : two-phase
Relative mole fraction of liquid phase : 0.6618
 Relative mole fraction of vapor phase : 0.3382
         Mole fraction in liquid phase : [0.41860774 0.13129475 0.45009751]
          Mole fraction in vapor phase : [0.95488922 0.03877265 0.00633814]
                              K values : [2.28110682 0.29530994 0.01408169]


PT Flashの結果、この温度・条件下では二相状態となることがわかる．全モル数の約2/3が液相に存在している．また、気相は95%がC1で、液相にはC10が多く存在する．

### ケース２：C1-C4-C10-CO2系

モル分率を下記のとおりとする．

- C1: 0.3
- C4: 0.05
- C10: 0.15
- CO2: 0.5

これは、モル分率のとおりケース１の系に同モル量CO2を添加したケースに相当する．

In [6]:
feed = [methane, butane, decane, co2]
Zi = np.array([0.3, 0.05, 0.15, 0.5])

kik = np.array([[0     , 0.0133, 0.0422, 0.0919],
                [0.0133, 0     , 0     , 0.1333],
                [0.0422, 0     , 0     , 0.1141],
                [0.0919, 0.1333, 0.1141, 0     ]
                ])

m = Mixture(feed, Zi)
m.set_BIPs(kik)
m.PT_flash(P,T,verbose=True)

PT Flash calculation converged.

                       Feed components : ['methane', 'butane', 'decane', 'co2']
                    Feed mole fraction : [0.3  0.05 0.15 0.5 ]

PT Flash at 355.4 degK and 137.9 BarA.

                                 Phase : two-phase
Relative mole fraction of liquid phase : 0.4458
 Relative mole fraction of vapor phase : 0.5542
         Mole fraction in liquid phase : [0.19384671 0.07637189 0.32257227 0.40720913]
          Mole fraction in vapor phase : [0.38537628 0.02878979 0.01120468 0.57462925]
                              K values : [1.98804587 0.37696845 0.03473543 1.41113995]


PT Flashの結果、この系でも二相状態となることがわかる．CO2を添加した結果、前のケースに比べて気相のモル数が増加している．炭化水素のK値も変化している．

*End of notebook...*